In [20]:
import requests
from bs4 import BeautifulSoup
import json
import time
from tqdm import tqdm
import re
import tibetcm_utils

import os
import json

In [27]:
class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.string  # or obj.text, depending on what you want to extract
        return super().default(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(e)

def read_json(path, file_name):
    """
    
    """
    with open(path+file_name, 'r') as openfile:
        # Reading from json file
        Loaded_file = json.load(openfile)
        print(f"Successfully loaded: {file_name}")

    return Loaded_file


In [29]:
def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    start = 1
    all_article = {}
    
    for i in tqdm(range(start, Total_lenght+1)):
        page_key = page_key_code + str(i)
        all_link_page = All_links_data[page_key]["Links"]
        page_article_index = 1
        
        for each_URL in all_link_page:
            article_content = tibetcm_utils.scrape_tibetcm_article_content(each_URL, tags=page_key_code)
            article_key = page_key + "_tibetcm_Article_" + str(page_article_index)
            page_article_index += 1
            all_article[article_key] = article_content


    Failure_count = 0
    for each_article_key in all_article:
        if all_article[each_article_key]["Response"] != 200:
            print(each_article_key)
            Failure_count += 1
    print(f"total Failure in the {page_key_list[1]} article: {Failure_count}")

    save_file_name = f"tibetcm_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/all_content/"
    save_json(path, save_file_name, all_article)
    return all_article
    
    

In [26]:
def get_json_files(directory):
    json_files = []
    i = 0
    # Loop through all files in the given directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            
            # Read and parse the JSON file
            with open(file_path, 'r') as file:
                try:
                    i += 1
                    All_links_data = json.load(file)
                    Total_lenght = len(All_links_data)
                    print(f"Total page in {filename}: {Total_lenght}")
                
                    print(f"page key name: {list(All_links_data.keys())[-1]}")
                    page_key_list = list(All_links_data.keys())[-1].split(" ")
                    
                    page_key_code = "Page "+page_key_list[1]+" "
                    print(f"Page key code: {page_key_code}")

                    final_file = get_content(All_links_data, Total_lenght, page_key_code, page_key_list)

                    # json_files.append((filename, json_data))
                except json.JSONDecodeError:
                    print(f"Error decoding JSON in file: {filename}")
        break
    print(i)
    # return json_files

# Example usage
directory_path = './data/links/'
json_files = get_json_files(directory_path)

Total page in tibetcm_ALL_link_ལེགས་རྩོམ།.json: 1
page key name: Page ལེགས་རྩོམ། 1
Page key code: Page ལེགས་རྩོམ། 


100%|██████████| 1/1 [01:25<00:00, 85.75s/it]

total Failure in the ལེགས་རྩོམ། article: 0
tibetcm_ALL_content_ལེགས་རྩོམ།.json


NameError: name 'Tag' is not defined

In [31]:
import json
from bs4 import Tag
from tqdm import tqdm
import tibetcm_utils
import os

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()  # Get all text from the tag
        return str(obj)  # Convert any other non-serializable object to string

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

def read_json(path, file_name):
    try:
        with open(path+file_name, 'r', encoding='utf-8') as openfile:
            Loaded_file = json.load(openfile)
            print(f"Successfully loaded: {file_name}")
        return Loaded_file
    except Exception as e:
        print(f"Error loading {file_name}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    start = 1
    all_article = {}
    
    for i in tqdm(range(start, Total_lenght+1)):
        page_key = page_key_code + str(i)
        all_link_page = All_links_data[page_key]["Links"]
        page_article_index = 1
        
        for each_URL in all_link_page:
            try:
                article_content = tibetcm_utils.scrape_tibetcm_article_content(each_URL, tags=page_key_code)
                article_key = page_key + "_tibetcm_Article_" + str(page_article_index)
                page_article_index += 1
                all_article[article_key] = article_content
            except Exception as e:
                print(f"Error scraping {each_URL}: {str(e)}")
    
    Failure_count = 0
    for each_article_key in all_article:
        if all_article[each_article_key]["Response"] != 200:
            print(each_article_key)
            Failure_count += 1
    print(f"total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"tibetcm_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/all_content/"
    save_json(path, save_file_name, all_article)

def get_json_files(directory):
    json_files = []
    i = 0
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    i += 1
                    All_links_data = json.load(file)
                    Total_lenght = len(All_links_data)
                    print(f"Total page in {filename}: {Total_lenght}")
                
                    print(f"page key name: {list(All_links_data.keys())[-1]}")
                    page_key_list = list(All_links_data.keys())[-1].split(" ")
                    
                    page_key_code = "Page "+page_key_list[1]+" "
                    print(f"Page key code: {page_key_code}")
                    get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {filename}: {str(e)}")
            except Exception as e:
                print(f"Error processing file {filename}: {str(e)}")
    print(f"Processed {i} files")

# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tibetcm_ALL_link_ལེགས་རྩོམ།.json: 1
page key name: Page ལེགས་རྩོམ། 1
Page key code: Page ལེགས་རྩོམ། 


100%|██████████| 1/1 [01:20<00:00, 80.99s/it]


total Failure in the ལེགས་རྩོམ། article: 0
tibetcm_ALL_content_ལེགས་རྩོམ།.json
Successfully saved: tibetcm_ALL_content_ལེགས་རྩོམ།.json
Total page in tibetcm_ALL_link_གཞུང་།.json: 1
page key name: Page གཞུང་། 1
Page key code: Page གཞུང་། 


100%|██████████| 1/1 [00:11<00:00, 11.55s/it]


total Failure in the གཞུང་། article: 0
tibetcm_ALL_content_གཞུང་།.json
Successfully saved: tibetcm_ALL_content_གཞུང་།.json
Total page in tibetcm_ALL_link_བསྟོད་ཚོགས།.json: 3
page key name: Page བསྟོད་ཚོགས། 3
Page key code: Page བསྟོད་ཚོགས། 


...

total Failure in the མི་རབས་གསུམ་པ། article: 0
tibetcm_ALL_content_མི་རབས་གསུམ་པ།.json
Successfully saved: tibetcm_ALL_content_མི་རབས་གསུམ་པ།.json
Total page in tibetcm_ALL_link_མི་རབས་བཞི་བ།.json: 1
page key name: Page མི་རབས་བཞི་བ། 1
Page key code: Page མི་རབས་བཞི་བ། 


100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


total Failure in the མི་རབས་བཞི་བ། article: 0
tibetcm_ALL_content_མི་རབས་བཞི་བ།.json
Successfully saved: tibetcm_ALL_content_མི་རབས་བཞི་བ།.json
Total page in tibetcm_ALL_link_ཚོང་བརྡ།.json: 1
page key name: Page ཚོང་བརྡ། 1
Page key code: Page ཚོང་བརྡ། 


100%|██████████| 1/1 [00:17<00:00, 17.16s/it]

total Failure in the ཚོང་བརྡ། article: 0
tibetcm_ALL_content_ཚོང་བརྡ།.json
Successfully saved: tibetcm_ALL_content_ཚོང་བརྡ།.json
Processed 71 files
